In [ ]:
import warnings
warnings.simplefilter("ignore")

import logging
logging.basicConfig(level=logging.INFO)

import os
import ast
import random
import smart_open
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from joblib import Parallel, delayed

from collections import OrderedDict, Counter
from tqdm import tqdm_notebook as tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.fasttext import FastText
from gensim.utils import tokenize


import sys
sys.path.append('..')

In [ ]:
file = "../../../Starspace/data/oms/text/oms-all_raw_preprocessed.txt"

In [ ]:
class FTextIter(object):
    def __init__(self, file_path):
        super(FTextIter, self).__init__()
        self.file_path = file_path

    def __iter__(self):
        with smart_open.smart_open(self.file_path, 'r', encoding='utf-8') as fin:
            for line in fin:
                yield list(tokenize(line))

                
def fasttext_generator(fname):
    
    prefix, file = os.path.split(fname)
    name = "oms_fasttext_300.model"
    filename = "{}/fastext/".format(prefix, name)

    if not os.path.isfile(filename):
        moo = FastText(size=300, window=3, min_count=1) # hs=0, negative=0, size=300
        moo.build_vocab(sentences=FTextIter(fname))
        total_examples = moo.corpus_count
        moo.train(sentences=FTextIter(fname), total_examples=total_examples, epochs=5)
        moo.save(filename)
    else:
        moo = FastText.load(filename)

    return moo

In [ ]:
model = fasttext_generator(file)